In [2]:
import pandas as pd
import numpy as np
import time
import lyricsgenius
import getpass
from tqdm import tqdm


### Extract lyrics for all songs in our dataframe

#### Setup access to Genius API using lyricsgenius

In [3]:
accesstoken = getpass.getpass()

········


In [4]:
client_access_token = accesstoken
genius = lyricsgenius.Genius(client_access_token, remove_section_headers=True,
                 skip_non_songs=True, excluded_terms=["Remix", "Live", "Edit", "Mix", "Club"])

In [5]:
songs = pd.read_pickle("../../files/pickle/audioanalysis.pkl")

In [6]:
songs.head()

id                             artist             title  year  month  week  \
0   1              Kollegah & Farid Bang   One Night Stand  2018      1     1   
1   2                              Bausa               FML  2018      1     1   
2   3                               Jazn            Bombay  2018      1     1   
3   4                         Tom Walker  Leave A Light On  2018      1     1   
4   5  G-Eazy feat. A$AP Rocky & Cardi B          No Limit  2018      1     1   

   totalweeksincharts  total2018weeksincharts  total2019weeksincharts  \
0                   4                       4                       0   
1                   7                       7                       0   
2                   9                       9                       0   
3                  28                      28                       0   
4                   5                       5                       0   

   total2020weeksincharts  ...  instrumentalness  liveness  valence    tempo  \
0                       0  ...          0.000000    0.1670    0.309  108.883   
1                       0  ...          0.000309    0.1640    0.246  108.589   
2                       0  ...          0.000167    0.0994    0.422   96.032   
3                       0  ...          0.000002    0.1330    0.267   68.976   
4                       0  ...          0.000000    0.0853    0.405  175.957   

   duration_ms  time_signature              spotify-id  \
0       163773               4  7J82QBqJjwbIDtm1Oxc0uo   
1       179718               4  4OwriBNZjK7VtYBxngT3OJ   
2       193040               4  083GGYAqJqUkRsy8JyLeyG   
3       185863               4  6lOWoTqVnAWXchddtTH31W   
4       245387               4  2DQ1ITjI0YoLFzuADN1ZBW   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/7J82QBqJjwbI...   
1  https://api.spotify.com/v1/tracks/4OwriBNZjK7V...   
2  https://api.spotify.com/v1/tracks/083GGYAqJqUk...   
3  https://api.spotify.com/v1/tracks/6lOWoTqVnAWX...   
4  https://api.spotify.com/v1/tracks/2DQ1ITjI0YoL...   

                                        analysis_url  \
0  https://api.spotify.com/v1/audio-analysis/7J82...   
1  https://api.spotify.com/v1/audio-analysis/4Owr...   
2  https://api.spotify.com/v1/audio-analysis/083G...   
3  https://api.spotify.com/v1/audio-analysis/6lOW...   
4  https://api.spotify.com/v1/audio-analysis/2DQ1...   

                                    uri  
0  spotify:track:7J82QBqJjwbIDtm1Oxc0uo  
1  spotify:track:4OwriBNZjK7VtYBxngT3OJ  
2  spotify:track:083GGYAqJqUkRsy8JyLeyG  
3  spotify:track:6lOWoTqVnAWXchddtTH31W  
4  spotify:track:2DQ1ITjI0YoLFzuADN1ZBW  

[5 rows x 27 columns]

#### Create list with search terms and use them to search for lyrics on genius.com

In [7]:
artisttweaked = [a.split(" feat. ")[0].split(" x ")[0].replace(" & ", " ").replace(" / ", " ").replace("'", "").replace(" [DE]", "").replace(".", " ").replace("?", "").replace("¿", "").replace(" +", "").replace(" [UK]", "").replace(" [2019]", "").replace(" [Live]", "") for a in songs['artist']]
titletweaked = [t.split(" feat. ")[0].split(" x ")[0].replace(" & ", " ").replace(" / ", " ").replace("'", " ").replace(" [DE]", "").replace(".", " ").replace("?", "").replace("¿", "").replace(" +", "").replace(" [UK]", "").replace(" [2019]", "").replace(" [Live]", "") for t in songs['title']]

searchterms = []
for a, t in zip(artisttweaked, titletweaked):
    s = a + " " + t
    searchterms.append(s)

In [8]:
lyrics = []

for searchy in tqdm(searchterms):
    try:
        songsind = genius.search_songs(searchy)
        url = songsind['hits'][0]['result']['url']
        song_lyrics = genius.lyrics(url)
        lyrics.append(song_lyrics)
    except:
        lyrics.append(np.nan)

100%|████████████████████████████████████████████████████████████████████████████| 1897/1897 [1:35:26<00:00,  3.02s/it]


In [9]:
lyrics.count(np.nan)

39

In [10]:
len(lyrics)

1897

In [11]:
songs['lyrics'] = lyrics

In [12]:
songs.columns.tolist()

['id',
 'artist',
 'title',
 'year',
 'month',
 'week',
 'totalweeksincharts',
 'total2018weeksincharts',
 'total2019weeksincharts',
 'total2020weeksincharts',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature',
 'spotify-id',
 'track_href',
 'analysis_url',
 'uri',
 'lyrics']

#### Save appended dataframe

In [13]:
songs[['id', 'artist', 'title', 'year', 'month', 'week', 'totalweeksincharts', 'total2018weeksincharts', 'total2019weeksincharts', 'total2020weeksincharts', 'lyrics']].to_csv("../../files/csv/lyrics.csv")
songs[['id', 'artist', 'title', 'year', 'month', 'week', 'totalweeksincharts', 'total2018weeksincharts', 'total2019weeksincharts', 'total2020weeksincharts', 'lyrics']].to_pickle("../../files/pickle/lyrics.pkl")